#Weed Detection using YOLO - V3 


---






## setting up environment

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (build)
!make

In [1]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

### mounting google drive for data 

In [ ]:
# mounting google drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

- From [Github](https://) copy all files from **traning** folder and paste it in your drive with folder name **weed_detection**.

In [ ]:
!ls /mydrive/projets/weed_detection

### Downloading dataset from kaggle

Please follow the steps below to download and use kaggle data within Google Colab:

1. Go to you kaggle > account, Scroll to API section and Click Expire API Token(if you have created previously)  to remove previous tokens

2. Click on Create New API Token - It will download kaggle.json file on your machine.

3. Now just run bellow cell.


- Link to [Dataset](https://www.kaggle.com/ravirajsinh45/crop-and-weed-detection-data-with-bounding-boxes)

In [ ]:
# instaling kaggle api
!pip install kaggle

In [ ]:
#upload kaggle.json file which you downloaded earlier
from google.colab import files
files.upload()

In [ ]:
# Some changes and permission in cloud
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ravirajsinh45/crop-and-weed-detection-data-with-bounding-boxes

In [ ]:
%cd darknet

In [ ]:
# unzip the zip file and its contents should now be in path /darknet/data/obj
!unzip ../crop-and-weed-detection-data-with-bounding-boxes.zip -d data/

In [ ]:
# uploading coustom .cfg, obj.names, obj.data, generate_train.py files from google drive
!cp /mydrive/projects/weed_detection/crop_weed.cfg ./cfg
!cp /mydrive/projects/weed_detection/obj.names ./data
!cp /mydrive/projects/weed_detection/obj.data  ./data
!cp /mydrive/projects/weed_detection/train.py ./


In [ ]:
# running python script
!python train.py

- Downloading pretrained weights from darknet


In [ ]:
# upload pretrained convolutional layer weights 
!wget http://pjreddie.com/media/files/darknet53.conv.74

- press ctrl+shift+i than paste below code 
- open console and paste below code else your runtime will crash after some time
```
    function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-toolbar-button#connect").click() 
    }
    setInterval(ClickConnect,60000)
```




# Training YOLO 

In [ ]:
# train your custom detector
!./darknet detector train data/obj.data cfg/crop_weed.cfg darknet53.conv.74 -dont_show

In [ ]:
imShow('chart.png')

# Testing

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=32/batch=1/' crop_weed.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' crop_weed.cfg
%cd ..

In [ ]:
!ls /mydrive/projects/weed_detection/test

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/crop_weed.cfg /mydrive/projects/weed_detection/backup/yolov3_custom_final.weights /mydrive/projects/weed_detection/test/weed_1.jpeg  -thresh 0.3
imShow('predictions.jpg')